# Elementos basicos de un Mock

Esta metodo de unittest simula objetos y sus metdodos.

Para utilizarlo:

1. Importarlo de unittest : `from unittest import mock`

2. El modo de uso implica usar el contexto patch de mock:
    - `with mock.patch("pandas.read_csv", return_value=mock_df) as mock_read_csv`
    - pandas.read_csv es la libreria y read_csv su metodo
    - return_value : hace referencia a lo que quieres que retorne el mock, en particular mock_df es otro mock previamente definido
    - as mock_read_csv : la variable donde sera retornado este mock.

3. Metodos tipicos de testeo:
    - `mock_read_csv.assert_called_once_with(<parameters>)` : Esto basicamente verifica que el mock retornado fue llamado con determinados parametros durante la ejecucion del codigo.

## Mocking un DataFrame

- Hacer el mock de un dataframe es tan trivial como hacer:

    - `with mock.patch('pandas.DataFrame', return_value=mock_df_instance) as mock_dataframe:`
    - Sin embargo si ya le queremos agregar un valor al mock ya es otro problema. Por ejemplo, al mismo yo le queria agregar este valor:

In [ ]:
# expected_data = np.array([
#             np.ravel(box.m),
#             np.ravel(box.arr_.x),
#             np.ravel(box.arr_.y),
#             np.ravel(box.arr_.z),
#             np.ravel(box.arr_.vx),
#             np.ravel(box.arr_.vy),
#             np.ravel(box.arr_.vz),
#         ]).T

- Ahora lo que sucede es que este valor se le asigna a mi mock a travez de la llamada a la funcion que quiero mockear. En contexto, dicha funcion es la siguiente:

In [ ]:
# def funcion(PARAMETROS):
# df = pd.DataFrame(np.array([
#         np.ravel(box.m), #This is log of Msun
#         np.ravel(box.arr_.x),
#         np.ravel(box.arr_.y),
#         np.ravel(box.arr_.z),
#         np.ravel(box.arr_.vx),
#         np.ravel(box.arr_.vy),
#         np.ravel(box.arr_.vz),
#     ]).T)

#     df.columns = ["m", "x", "y", "z", "vx", "vy", "vz"]
#     df.to_csv(
#         file_path,
#         sep=" ",
#         index=False,
#         header=False,  # float_format="%.2f"
#     )

- Como puedo asegurar entonces que llame a mi dataframe con esos argumentos?
- Como puedo asegurar que llame al metodo to_csv de mi dataframe con determinados argumentos?

- En principio la respuesta parece facil, simplemente tendria que hacer: `mock_dataframe.assert_called_once_with(expected_data)`
- Bueno esto, produce un error, el cual es el siguiente: ` FAILED ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()`
- La razon: *The error message you are encountering (ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()) is likely due to how np.ravel interacts with the assert_called_once_with method during the comparison of arrays. When you use np.ravel, it flattens the array, but the actual mock_dataframe call is comparing arrays in an ambiguous context (i.e., it compares whole arrays as if they were booleans, which causes this error).*

- La solucion? Utilizar el siguiente metodo del mock: `args, kwargs = mock_dataframe.call_args`. Explicacion: *In the context of mock.patch and unittest.mock, mock_dataframe.call_args provides the arguments that were passed to the mock function when it was called. The first element is a tuple of positional arguments (args). The second element is a dictionary of keyword arguments (kwargs).*

- Verificamos que efectivamente esos fueron los argumentos: `assert np.array_equal(args[0], expected_data)  # Check data equality`

- IMPORTANTE, ese args[0] es clave si no va a fallar la prueba. Por otro lado si utilizas ipdb para ver cuales son los valores de args o args[0] no vas a ver ningun output.

- Finalmente verificamos la llamada to_csv: `mock_df_instance.to_csv.assert_called_once_with(`
        `mock_file_path,`
        `sep=" ",`
        `index=False,`
        `header=False`
    `)`

# 1 Mock objetos

## Mock un metodo de una clase

- contexto: tengo una clase PopCorn que hereda de SVFPopCorn, dentro de PopCorn utilizo super().model_find(box) que es un metodo de SVFPopCorn. La idea es poder mockear esto, para hacer el test

- Para mockear el metodo de una clase se aconseja usar: mock.patch.object(< clase>, 'metodo', return_value=mock)

- Esto mockeara solamente SVFPopCorn.model_find()


# 2 Explorar un mock

Utilizanod ipdb podemos explorar que sucede con el mock, por ejemplo si tenemos

- with mock.patch('some.module') as p:

si tenemos la consola de ipdb abierta podemos hacer : 

- p.call_args : muestra los argumentos con los que se llamo el mock
- p.return_value : muestra el valor retornado por el mock
- p.mock_calls : llamadas al mock
- dir(p) : lista de atributos del mock

Aqui hay una lista de otros atributos del mock:
'assert_any_call', 'assert_called', 'assert_called_once', 'assert_called_once_with', 'assert_called_with', 'assert_has_calls', 'assert_not_called', 'attach_mock', 'call_args', 'call_args_list', 'call_count', 'called', 'configure_mock', 'method_calls', 'mock_add_spec', 'mock_calls', 'reset_mock', 'return_value', 'side_effect'

## Hacer que un mock retorne varios valores

- Si por ejemplo, lo que queremos que un determinado mock, retorne 3 valores, podemos utilijar mock.return_value:

-   mock_of_function.return_value = [ mock.MagickMock, value2, value3] etc

# Multiple Mocking

- Para hacer esto utilizamos: mock.patch.multiple, como en el siguiente ejemplo:

- with mock.patch.multiple(
                "voidfindertk.svf_popcorn._svf_pc_postprocessing",
                get_void_properties=mock.DEFAULT,
                get_tracers_in_voids=mock.DEFAULT,
            ) as svf_pc_postprocessing_mocks:

- cuando hacemos esto, si no mal recuerdo la unica opcion que habia era utilizar mock.DEFAULT
- Observe que estamos metiendo todas las opciones en una sola variable svf_pc_postprocessing_mocks
- Ahora para probar asersiones sobre cada uno de estos elementos, hacemos los siguiente:

- svf_pc_postprocessing_mocks["get_void_properties"].assert_called_once_with(
        popcorn_output_file_path=str(run_work_dir / pn.SPHFILE)
    )

- Basicamente estamos llamando a la variable donde habiamos guardado nuestros mock, y a traves de llamar especificamente con el string asociado al mock especifico podremos hacer referencia a este mock en especifico.

# Verificar que a una variable mockeada se le asigno un valor

Contexto, estoy mockeando un objeto, el cual retorna una variable: ```model_find_parameters``` . Esta variable sera un mock tambien. Lo que sucede posteriormente es que en algun momento del codigo se llama a esta variable y se le asigna un valor: ```model_find_parameters["build_popcorn"] = True``` . Observese que concretamente esta no es el mismo mock que model_find_parameters. La pregunta es, como puedo preguntar que efectivamente llame al mock model_find_parameters["build_popcorn"] y le asigne la variable True?

Respuesta:

model_find_parameters.__setitem__.assert_called_once_with("build_popcorn", True)

- Tener en cuenta que la sintaxys de .assert_called_once_with es : .assert_called_once_with(args)